In [1]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np

In [2]:
# setting device on GPU if available, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [3]:
def generate_samples(n, train_split):
    np.random.seed(42)
    X = np.random.uniform(-1, 1, (n, 1))
    train_N = int(n * train_split)
    y = np.sin(3 * np.pi * X) + 0.1 * np.random.randn(n, 1)
    X_train, y_train = X[:train_N], y[:train_N]
    X_test, y_test = X[train_N:], y[train_N:]
    
    X_train = torch.from_numpy(X_train).float().to(device)
    y_train = torch.from_numpy(y_train).float().to(device)
    X_test = torch.from_numpy(X_test).float().to(device)
    y_test = torch.from_numpy(y_test).float().to(device)

    return X_train, X_test, y_train, y_test

In [4]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(1, 1000)
        self.fc2 = torch.nn.Linear(1000, 1000)
        self.fc3 = torch.nn.Linear(1000, 1)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [5]:
def train_and_evaluate_model(model, sample_size, optimizer, criterion):
    X_train, X_test, y_train, y_test = generate_samples(sample_size, 0.8)
    train_losses = []
    test_losses = []
    for epoch in range(500):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())

        with torch.no_grad():
            outputs = model(X_test)
            test_loss = criterion(outputs, y_test)
            test_losses.append(test_loss.item())

    return test_losses[-1]

In [ ]:
sample_sizes = [i for i in range (10, 3000)]
test_mses = []
for sample_size in sample_sizes:
    model = Net()
    model = model.to(device)
    optimizer =  torch.optim.SGD(model.parameters(), lr=0.01)
    criterion = torch.nn.MSELoss()
    test_mse = train_and_evaluate_model(model, sample_size, optimizer, criterion)
    test_mses.append(test_mse)

In [ ]:
plt.plot(sample_sizes, test_mses)
plt.xlabel('No. Samples')
plt.ylabel('Test MSE')
plt.show()
